# Building AI Agents with OpenAI Assistants API and Google Tools

In this lesson, we'll explore how to create an AI agent using OpenAI's Assistants API with Google tool capabilities (Gmail and Google Calendar). By the end of this tutorial, you'll have a working AI agent that can read emails, send emails, and manage calendar events through natural language commands.

## 1. Introduction to OpenAI Assistants API

The OpenAI Assistants API is designed to help developers build powerful AI assistants that can perform a variety of tasks. Unlike the regular Chat Completions API, the Assistants API provides a more structured way to build conversational agents with memory and tool-calling capabilities.

### Key Concepts

- **Assistant**: A purpose-built AI that can use tools and provide responses
- **Thread**: A conversation session between an Assistant and a user
- **Message**: Text or file content created by an Assistant or user
- **Run**: An invocation of an Assistant on a Thread
- **Run Step**: A detailed record of steps the Assistant takes during a Run
- **Tools**: Capabilities like Code Interpreter, File Search, and Function Calling

## 2. Setting Up the Environment

Let's start by setting up our development environment with the necessary dependencies.

In [ ]:
# Install required packages
!pip install openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client python-dotenv

Create a basic project structure:

In [ ]:
assistants-google-agent/
├── .env                    # Environment variables
├── assistants.py           # Main Assistant implementation
├── tools.py                # Google API tools
├── function_schemas/       # Function definitions for Assistants API
│   ├── gmail_schema.json
│   └── calendar_schema.json
├── credentials.json        # Google API credentials
└── requirements.txt        # Dependencies

Set up environment variables in `.env`:

In [ ]:
# .env file content
import os
from dotenv import load_dotenv

# Create or modify the .env file
with open('.env', 'w') as f:
    f.write('''OPENAI_API_KEY=your_openai_api_key_here
GOOGLE_APPLICATION_CREDENTIALS=credentials.json
''')

# Load environment variables
load_dotenv()

## 3. Google API Setup and Authentication

To use Google services like Gmail and Calendar, we need to set up authentication with the Google API.

### 3.1 Google Cloud Project Setup

1. Go to [Google Cloud Console](https://console.cloud.google.com)
2. Create a new project or select an existing one
3. Enable Gmail API and Google Calendar API
4. Create OAuth2 credentials (Web application or Desktop app type)
5. Download the credentials JSON file and save it as `credentials.json`

### 3.2 Authentication Code

Let's create a script to handle Google API authentication:

In [ ]:
import os
import json
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# Define the necessary scopes
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/calendar'
]

def get_google_credentials():
    """Get and refresh Google API credentials."""
    creds = None
    
    # Check if token.json exists
    if os.path.exists('token.json'):
        with open('token.json', 'r') as token:
            creds = Credentials.from_authorized_user_info(json.load(token), SCOPES)
    
    # If credentials don't exist or are invalid
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save the credentials for future use
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return creds

# Run this once to authenticate and generate token.json
if __name__ == "__main__":
    credentials = get_google_credentials()
    print("Authentication successful!")

Run this script once to generate the `token.json` file, which will be used for subsequent API calls.

## 4. Building Tool Functions

Now, let's create the functions that will interface with Gmail and Google Calendar APIs. These will be used as tools by our Assistant.

### 4.1 Google Service Wrapper

First, let's create a utility class to manage Google service connections:

In [ ]:
# tools.py
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import json
import base64
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta
from typing import List, Dict, Optional

# Define the necessary scopes
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/calendar'
]

class GoogleServices:
    _instance = None
    _gmail = None
    _calendar = None
    _credentials = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(GoogleServices, cls).__new__(cls)
            cls._instance._initialize()
        return cls._instance

    def _initialize(self):
        """Initialize credentials and services"""
        self._credentials = self._get_credentials()
        self._gmail = build('gmail', 'v1', credentials=self._credentials)
        self._calendar = build('calendar', 'v3', credentials=self._credentials)

    def _get_credentials(self):
        """Get and refresh credentials if needed"""
        creds = None
        if os.path.exists('token.json'):
            with open('token.json', 'r') as token:
                creds = Credentials.from_authorized_user_info(json.load(token), SCOPES)
        
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
        
        return creds

    @property
    def gmail(self):
        """Get Gmail service"""
        return self._gmail

    @property
    def calendar(self):
        """Get Calendar service"""
        return self._calendar

# Create a global instance
services = GoogleServices()

### 4.2 Gmail Functions

Next, let's implement Gmail functions for reading and sending emails:

In [ ]:
# Continuing in tools.py

def read_emails(query: str = "in:inbox", max_results: int = 10) -> List[Dict]:
    """Read emails from Gmail using Gmail API."""
    try:
        results = services.gmail.users().messages().list(
            userId='me', q=query, maxResults=max_results).execute()
        messages = results.get('messages', [])

        emails = []
        for message in messages:
            msg = services.gmail.users().messages().get(
                userId='me', id=message['id']).execute()
            
            headers = msg['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), 'No Subject')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), 'Unknown')
            date = next((h['value'] for h in headers if h['name'] == 'Date'), '')
            
            # Extract body content
            body = ""
            if 'parts' in msg['payload']:
                parts = msg['payload']['parts']
                for part in parts:
                    if part['mimeType'] == 'text/plain':
                        if 'data' in part['body']:
                            body = base64.urlsafe_b64decode(part['body']['data']).decode()
                        break
            else:
                if 'data' in msg['payload']['body']:
                    body = base64.urlsafe_b64decode(msg['payload']['body']['data']).decode()

            emails.append({
                'id': message['id'],
                'subject': subject,
                'sender': sender,
                'date': date,
                'body': body[:500] + ('...' if len(body) > 500 else '')  # Truncate long emails
            })

        return emails
    except Exception as e:
        return [{'error': str(e)}]

def send_email(to: str, subject: str, body: str) -> Dict:
    """Send an email using Gmail API."""
    try:
        message = MIMEMultipart()
        message['to'] = to
        message['from'] = "me"
        message['subject'] = subject
        
        msg = MIMEText(body)
        message.attach(msg)
        
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
        services.gmail.users().messages().send(
            userId='me', body={'raw': raw}).execute()
        
        return {
            "success": True,
            "message": "Email sent successfully!"
        }
    except Exception as e:
        return {
            "success": False,
            "message": f"Error: {str(e)}"
        }

### 4.3 Google Calendar Functions

Now, let's implement Google Calendar functions:

In [ ]:
# Continuing in tools.py

def list_events(max_results: int = 10, time_min: Optional[str] = None, 
                time_max: Optional[str] = None, calendar_id: str = 'primary') -> List[Dict]:
    """List events from Google Calendar."""
    try:
        # Default time range if not specified
        if not time_min:
            time_min = datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        if not time_max:
            time_max = (datetime.utcnow() + timedelta(days=7)).isoformat() + 'Z'
            
        events_result = services.calendar.events().list(
            calendarId=calendar_id, 
            timeMin=time_min,
            timeMax=time_max, 
            maxResults=max_results,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        formatted_events = []
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            end = event['end'].get('dateTime', event['end'].get('date'))
            
            formatted_event = {
                'id': event['id'],
                'summary': event.get('summary', 'No title'),
                'start': start,
                'end': end,
                'location': event.get('location', ''),
                'description': event.get('description', ''),
                'attendees': [attendee.get('email') for attendee in event.get('attendees', [])]
            }
            formatted_events.append(formatted_event)
            
        return formatted_events
        
    except Exception as e:
        return [{'error': str(e)}]

def create_event(summary: str, start_time: str, end_time: str, description: str = '',
                location: str = '', attendees: Optional[List[str]] = None, 
                calendar_id: str = 'primary') -> Dict:
    """Create a new event in Google Calendar."""
    try:
        event = {
            'summary': summary,
            'location': location,
            'description': description,
            'start': {
                'dateTime': start_time,
                'timeZone': 'UTC',
            },
            'end': {
                'dateTime': end_time,
                'timeZone': 'UTC',
            }
        }
        
        if attendees:
            event['attendees'] = [{'email': email} for email in attendees]
            
        event = services.calendar.events().insert(calendarId=calendar_id, body=event).execute()
        
        return {
            'success': True,
            'event_id': event['id'],
            'html_link': event['htmlLink'],
            'message': 'Event created successfully!'
        }
        
    except Exception as e:
        return {
            'success': False,
            'message': f'Error: {str(e)}'
        }

def delete_event(event_id: str, calendar_id: str = 'primary') -> Dict:
    """Delete an event from Google Calendar."""
    try:
        services.calendar.events().delete(calendarId=calendar_id, eventId=event_id).execute()
        
        return {
            'success': True,
            'message': 'Event deleted successfully!'
        }
        
    except Exception as e:
        return {
            'success': False,
            'message': f'Error: {str(e)}'
        }

### 4.4 Function Schemas for the Assistant

OpenAI's Assistants API requires function schemas to be defined for tool calling. Let's create these schema files:

In [ ]:
# Create function schemas directory
import os
import json

os.makedirs('function_schemas', exist_ok=True)

# Create Gmail function schemas
gmail_read_schema = {
    "name": "read_emails",
    "description": "Read emails from Gmail using Gmail API",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Gmail search query (e.g., 'in:inbox', 'from:someone@example.com')",
                "default": "in:inbox"
            },
            "max_results": {
                "type": "integer",
                "description": "Maximum number of results to return",
                "default": 10
            }
        }
    }
}

gmail_send_schema = {
    "name": "send_email",
    "description": "Send an email using Gmail API",
    "parameters": {
        "type": "object",
        "properties": {
            "to": {
                "type": "string",
                "description": "Recipient email address"
            },
            "subject": {
                "type": "string",
                "description": "Email subject"
            },
            "body": {
                "type": "string",
                "description": "Email body content"
            }
        },
        "required": ["to", "subject", "body"]
    }
}

# Create Calendar function schemas
calendar_list_schema = {
    "name": "list_events",
    "description": "Lists upcoming events from the user's Google Calendar",
    "parameters": {
        "type": "object",
        "properties": {
            "max_results": {
                "type": "integer",
                "description": "Maximum number of events to return (default: 10)"
            },
            "time_min": {
                "type": "string",
                "description": "Start time in ISO format (default: now)"
            },
            "time_max": {
                "type": "string",
                "description": "End time in ISO format (default: 7 days from now)"
            }
        }
    }
}

calendar_create_schema = {
    "name": "create_event",
    "description": "Creates a new event in the user's Google Calendar",
    "parameters": {
        "type": "object",
        "properties": {
            "summary": {
                "type": "string",
                "description": "Event title"
            },
            "start_time": {
                "type": "string",
                "description": "Start time in ISO format (e.g., '2025-03-15T09:00:00Z')"
            },
            "end_time": {
                "type": "string",
                "description": "End time in ISO format (e.g., '2025-03-15T10:00:00Z')"
            },
            "description": {
                "type": "string",
                "description": "Event description"
            },
            "location": {
                "type": "string",
                "description": "Event location"
            },
            "attendees": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "description": "List of attendee email addresses"
            }
        },
        "required": ["summary", "start_time", "end_time"]
    }
}

calendar_delete_schema = {
    "name": "delete_event",
    "description": "Deletes an event from the user's Google Calendar",
    "parameters": {
        "type": "object",
        "properties": {
            "event_id": {
                "type": "string",
                "description": "ID of the event to delete"
            }
        },
        "required": ["event_id"]
    }
}

# Write schemas to files
with open('function_schemas/read_email_schema.json', 'w') as f:
    json.dump(gmail_read_schema, f, indent=2)

with open('function_schemas/send_email_schema.json', 'w') as f:
    json.dump(gmail_send_schema, f, indent=2)

with open('function_schemas/list_events_schema.json', 'w') as f:
    json.dump(calendar_list_schema, f, indent=2)

with open('function_schemas/create_event_schema.json', 'w') as f:
    json.dump(calendar_create_schema, f, indent=2)

with open('function_schemas/delete_event_schema.json', 'w') as f:
    json.dump(calendar_delete_schema, f, indent=2)

## 5. Implementing the Assistant

Now, let's implement the core Assistant functionality that will use these tools.

### 5.1 Main Assistant Implementation

In [ ]:
# assistants.py
import os
import json
import time
from openai import OpenAI
from dotenv import load_dotenv
from tools import read_emails, send_email, list_events, create_event, delete_event

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def execute_function(function_name, arguments):
    """
    Execute a function based on the function name and provided arguments.
    """
    if function_name == 'read_emails':
        query = arguments.get("query", "in:inbox")
        max_results = arguments.get("max_results", 10)
        return read_emails(query=query, max_results=max_results)
    
    elif function_name == 'send_email':
        to = arguments.get("to")
        subject = arguments.get("subject")
        body = arguments.get("body")
        return send_email(to=to, subject=subject, body=body)
    
    elif function_name == 'list_events':
        max_results = arguments.get("max_results", 10)
        time_min = arguments.get("time_min")
        time_max = arguments.get("time_max")
        return list_events(max_results=max_results, time_min=time_min, time_max=time_max)
    
    elif function_name == 'create_event':
        summary = arguments.get("summary")
        start_time = arguments.get("start_time")
        end_time = arguments.get("end_time")
        description = arguments.get("description", "")
        location = arguments.get("location", "")
        attendees = arguments.get("attendees")
        return create_event(
            summary=summary, 
            start_time=start_time, 
            end_time=end_time,
            description=description, 
            location=location, 
            attendees=attendees
        )
    
    elif function_name == 'delete_event':
        event_id = arguments.get("event_id")
        return delete_event(event_id=event_id)
    
    else:
        return {"error": "Function not recognized"}

def create_assistant():
    """
    Create an Assistant with Google tools.
    """
    # Load function schemas
    with open('function_schemas/read_email_schema.json') as f:
        read_email_schema = json.load(f)
    
    with open('function_schemas/send_email_schema.json') as f:
        send_email_schema = json.load(f)
    
    with open('function_schemas/list_events_schema.json') as f:
        list_events_schema = json.load(f)
    
    with open('function_schemas/create_event_schema.json') as f:
        create_event_schema = json.load(f)
    
    with open('function_schemas/delete_event_schema.json') as f:
        delete_event_schema = json.load(f)
    
    # Define the tools
    tools = [
        {"type": "function", "function": read_email_schema},
        {"type": "function", "function": send_email_schema},
        {"type": "function", "function": list_events_schema},
        {"type": "function", "function": create_event_schema},
        {"type": "function", "function": delete_event_schema}
    ]
    
    # Create the assistant
    assistant = client.beta.assistants.create(
        name="Google Services Assistant",
        instructions="You are a helpful assistant that can manage emails and calendar events. Use the available tools to help users read emails, send emails, and manage their calendar.",
        model="gpt-4o",
        tools=tools
    )
    
    return assistant

def process_thread_with_assistant(user_query, assistant_id, model="gpt-4o"):
    """
    Process a user query with the assistant.
    """
    try:
        # Create a thread
        thread = client.beta.threads.create()
        
        # Add a message to the thread
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_query
        )
        
        # Run the assistant
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant_id,
            model=model
        )
        
        # Wait for the run to complete
        while run.status in ["queued", "in_progress", "requires_action"]:
            # If the run requires action, execute the function
            if run.status == "requires_action":
                tool_outputs = []
                
                for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                    function_name = tool_call.function.name
                    arguments = json.loads(tool_call.function.arguments)
                    
                    # Execute the function
                    function_output = execute_function(function_name, arguments)
                    
                    # Add the output to tool_outputs
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": json.dumps(function_output)
                    })
                
                # Submit the tool outputs
                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )
            
            # Wait for a moment before checking again
            time.sleep(1)
            
            # Get the updated run
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
        
        # Get the messages
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        
        # Find the latest assistant message
        latest_message = None
        for message in messages.data:
            if message.role == "assistant":
                latest_message = message
                break
        
        # Return the assistant's response
        if latest_message:
            content_parts = []
            for content in latest_message.content:
                if content.type == "text":
                    content_parts.append(content.text.value)
            
            return " ".join(content_parts)
        else:
            return "No response from the assistant."
    
    except Exception as e:
        return f"Error: {str(e)}"

## 6. Running the Agent

Now, let's create a simple script to run and test our agent:

In [ ]:
# run.py
from assistants import create_assistant, process_thread_with_assistant

def main():
    # Create the assistant (do this once)
    assistant = create_assistant()
    assistant_id = assistant.id
    print(f"Assistant created with ID: {assistant_id}")
    
    # Save the assistant ID for future use
    with open('assistant_id.txt', 'w') as f:
        f.write(assistant_id)
    
    # Process user queries
    while True:
        user_query = input("\nEnter your query (or 'exit' to quit): ")
        
        if user_query.lower() == 'exit':
            break
        
        print("\nProcessing your query...")
        response = process_thread_with_assistant(user_query, assistant_id)
        print("\nAssistant response:")
        print(response)

if __name__ == "__main__":
    main()

Or, if you already have an assistant ID:

In [ ]:
# run_existing.py
from assistants import process_thread_with_assistant
import os

def main():
    # Load existing assistant ID
    if os.path.exists('assistant_id.txt'):
        with open('assistant_id.txt', 'r') as f:
            assistant_id = f.read().strip()
    else:
        assistant_id = input("Enter your assistant ID: ")
    
    # Process user queries
    while True:
        user_query = input("\nEnter your query (or 'exit' to quit): ")
        
        if user_query.lower() == 'exit':
            break
        
        print("\nProcessing your query...")
        response = process_thread_with_assistant(user_query, assistant_id)
        print("\nAssistant response:")
        print(response)

if __name__ == "__main__":
    main()

## 7. Sample Interactions

Here are some sample queries you can try with your assistant:

1. Reading emails:
   - "Show me my recent emails"
   - "Find emails from [specific sender]"
   - "Search for emails about [topic]"

2. Sending emails:
   - "Send an email to john@example.com with subject 'Meeting Tomorrow' and tell him I'll be there"
   - "Draft an email to my team about the project status"

3. Calendar management:
   - "What's on my calendar for next week?"
   - "Schedule a meeting with Sarah tomorrow at 2pm"
   - "Create a dentist appointment for March 15th at 10am"
   - "Delete my meeting with Marketing team"

## 8. Extensions and Next Steps

Here are some ways you could extend this project:

1. **Add more Google tools**:
   - Google Docs integration
   - Google Drive file management
   - Google Tasks for to-do lists

2. **Improve the user interface**:
   - Create a web interface using Flask or Streamlit
   - Implement a Slack bot (as shown in the original repo)

3. **Add memory and context**:
   - Store conversation history
   - Implement personalization based on user preferences

4. **Implement error handling and retries**:
   - Add more robust error handling
   - Implement rate limiting and retry mechanisms

5. **Security enhancements**:
   - Implement proper credential storage
   - Add user authentication

## Conclusion

In this lesson, we've built an AI agent using OpenAI's Assistants API with Google tool capabilities. This agent can read and send emails, and manage calendar events through natural language commands. You've learned:

1. How to set up and authenticate with Google APIs
2. How to implement tool functions for Gmail and Google Calendar
3. How to create and configure an OpenAI Assistant with tools
4. How to process user queries and execute functions based on the Assistant's decisions

The approach shown here can be extended to many other APIs and services, allowing you to create powerful AI agents tailored to your specific needs.

Happy building!